In [4]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 5.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("athletes.csv")
data['Total_lift'] = data['candj'] + data['snatch'] + data['deadlift'] + data['backsq']

# Remove not relevant columns
data = data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                               'train','background','experience','schedule','howlong', \
                               'deadlift','candj','snatch','backsq','experience',\
                               'background','schedule','howlong'])

data = data.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',\
                              'filthy50','fgonebad','run400','run5k','pullups','train'])

# Remove Outliers

data = data[data['weight'] < 1500]
data = data[data['gender'] != '--']
data = data[data['age'] >= 18]
data = data[(data['height'] < 96) & (data['height'] > 48)]

data = data[(data['deadlift'] > 0) & (data['deadlift'] <= 1105)|((data['gender'] == 'Female') \
                 & (data['deadlift'] <= 636))]
data = data[(data['candj'] > 0) & (data['candj'] <= 395)]
data = data[(data['snatch'] > 0) & (data['snatch'] <= 496)]
data = data[(data['backsq'] > 0) & (data['backsq'] <= 1069)]

# Clean Survey Data

decline_dict = {'Decline to answer|': np.nan}
data = data.replace(decline_dict)
data = data.dropna(subset=['background','experience','schedule','howlong','eat'])

data.shape

(30029, 15)

## H20

### All features

In [5]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

h2o.init()

hf = h2o.H2OFrame(data)

# target and features
target = "Total_lift"
features = [col for col in hf.columns if col != target]

# run AutoML
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=features, y=target, training_frame=hf)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplpprk1n5
  JVM stdout: /tmp/tmplpprk1n5/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmplpprk1n5/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_unknownUser_qaeelg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_1_20250720_144211


GLM Model: summary
    family    link      regularization              lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  --------  --------------------------  ------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------------------------------------------------------------------------------
    gaussian  identity  Ridge ( lambda = 0.01677 )  nlambda = 30, lambda.max = 16773.0, lambda.min = 0.01677, lambda.1se = 0.01677  263                           263                            30                      AutoML_1_20250720_144211_training_Key_Frame__upload_b2fcdd4e75c00cc64a28feb1dae24aa3.hex

ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 1.7762951987826243
RMSE: 1.3327772502495023
MAE: 1.0144987843119495
RMSLE: 0.016218537922479986
Mean Residual Deviance: 1.7762951987826243
R^2: 0.9999454030222316
Null degrees of freedom: 7162
Residual degrees of freedom: 6899
Null deviance: 233045912.5933236
Residual deviance: 12723.602508879938
AIC: 24973.070715654412

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 2.7903317047995464
RMSE: 1.6704285991324341
MAE: 1.2718587816434463
RMSLE: 0.018502199032524345
Mean Residual Deviance: 2.7903317047995464
R^2: 0.9999142351574457
Null degrees of freedom: 7162
Residual degrees of freedom: 6908
Null deviance: 233128806.9562735
Residual deviance: 19987.146001479152
AIC: 28190.100940456865

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     6042.87      106.851      5997.95       6050.37       5968.44       6224.89       5972.72
loglikelihood           0            0            0             0             0             0             0
mae                     1.27186      0.0312105    1.24513       1.28109       1.25442       1.32236       1.25631
mean_residual_deviance  2.79036      0.211869     2.69237       2.81223       2.64855       3.14994       2.64872
mse                     2.79036      0.211869     2.69237       2.81223       2.64855       3.14994       2.64872
null_deviance           4.66258e+07  2.682e+06    4.81296e+07   4.54785e+07   4.2432e+07    4.88948e+07   4.81938e+07
r2                      0.999914     6.02995e-06  0.99992       0.999911      0.999911      0.999908      0.999921
residual_deviance       3997.43      302.717      3858.17       4029.93       3795.37       4510.72       3792.96
rmse                    1.66951      0.0622517    1.64085       1.67697       1.62744       1.77481       1.62749
rmsle                   0.0132237    0.0144669    0.0353261     0.00270735    0.00141327    0.00631646    0.0203553

Scoring History: 
     timestamp            duration    iteration    lambda    predictors    deviance_train          deviance_xval       deviance_se          alpha    iterations    training_rmse       training_deviance    training_mae        training_r2
---  -------------------  ----------  -----------  --------  ------------  ----------------------  ------------------  -------------------  -------  ------------  ------------------  -------------------  ------------------  ------------------
     2025-07-20 14:42:54  0.000 sec   1            .17E5     264           32520.95798438803       32535.651829660306  840.5343592648243    0.0
     2025-07-20 14:42:54  0.006 sec   2            .1E5      264           32512.589012180346      32528.95529293135   840.3696158208695    0.0
     2025-07-20

In [10]:
# top 5 features
aml.leader.varimp(use_pandas=True).head(5)

,variable,relative_importance,scaled_importance,percentage
0,deadlift,61.797684,1.000000,0.289601
1,backsq,59.518085,0.963112,0.278918
2,candj,40.314522,0.652363,0.188925
3,snatch,35.780140,0.578988,0.167675
4,weight,0.879462,0.014231,0.004121


In [7]:
lb = aml.leaderboard
lb.as_data_frame().head(10)


/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,model_id,rmse,mse,mae,rmsle,mean_residual_deviance
0,GLM_1_AutoML_1_20250720_144211,1.670429,2.790332,1.271859,0.018502,2.790332
1,StackedEnsemble_BestOfFamily_1_AutoML_1_202507...,2.145513,4.603227,1.057022,0.027136,4.603227
2,StackedEnsemble_AllModels_1_AutoML_1_20250720_...,2.497829,6.239148,1.176252,0.031634,6.239148
3,XGBoost_3_AutoML_1_20250720_144211,11.646006,135.629455,5.207331,0.035801,135.629455
4,XGBoost_2_AutoML_1_20250720_144211,12.558779,157.722938,6.480968,0.042536,157.722938
5,GBM_2_AutoML_1_20250720_144211,13.590901,184.712602,5.833538,0.063935,184.712602
6,GBM_3_AutoML_1_20250720_144211,14.342705,205.713192,6.210447,0.062679,205.713192
7,XGBoost_1_AutoML_1_20250720_144211,15.221767,231.702181,6.952768,0.068354,231.702181
8,GBM_4_AutoML_1_20250720_144211,15.337525,235.239683,8.218005,0.063512,235.239683
9,XRT_1_AutoML_1_20250720_144211,17.205705,296.036280,7.884413,0.061248,296.036280


In [8]:
for model_id in aml.leaderboard['model_id'].as_data_frame().values[:10]:
    model = h2o.get_model(model_id[0])
    print(f"{model.model_id} - Training time (sec): {model._model_json['output']['run_time'] / 1000:.2f}")


/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


GLM_1_AutoML_1_20250720_144211 - Training time (sec): 0.41
StackedEnsemble_BestOfFamily_1_AutoML_1_20250720_144211 - Training time (sec): 5.79
StackedEnsemble_AllModels_1_AutoML_1_20250720_144211 - Training time (sec): 4.49
XGBoost_3_AutoML_1_20250720_144211 - Training time (sec): 5.88
XGBoost_2_AutoML_1_20250720_144211 - Training time (sec): 7.08
GBM_2_AutoML_1_20250720_144211 - Training time (sec): 1.14
GBM_3_AutoML_1_20250720_144211 - Training time (sec): 1.75
XGBoost_1_AutoML_1_20250720_144211 - Training time (sec): 3.51
GBM_4_AutoML_1_20250720_144211 - Training time (sec): 2.44
XRT_1_AutoML_1_20250720_144211 - Training time (sec): 8.93


### Top three features

In [11]:
h2o.init()

hf = h2o.H2OFrame(data)

# target and top 3 features only
target = "Total_lift"
top3_features = ["deadlift", "backsq", "candj"]

aml_top3 = H2OAutoML(max_models=10, seed=1)
aml_top3.train(x=top3_features, y=target, training_frame=hf)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,53 mins 53 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_unknownUser_qaeelg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.029 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_2_20250720_153556


GLM Model: summary
    family    link      regularization              lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  --------  --------------------------  ------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------------------------------------------------------------------------------
    gaussian  identity  Ridge ( lambda = 0.01677 )  nlambda = 30, lambda.max = 16773.0, lambda.min = 0.01677, lambda.1se = 0.02701  3                             3                              30                      AutoML_2_20250720_153556_training_Key_Frame__upload_b39a6ae042a4d6b04261d5aca67f4eac.hex

ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 235.46648185636312
RMSE: 15.344917134229274
MAE: 11.546316072320158
RMSLE: 0.0134917251469452
Mean Residual Deviance: 235.46648185636312
R^2: 0.9927626003358386
Null degrees of freedom: 7162
Residual degrees of freedom: 7159
Null deviance: 233045912.5933236
Residual deviance: 1686646.409537129
AIC: 59458.92913480968

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 236.65292080513183
RMSE: 15.383527580016615
MAE: 11.587143419147322
RMSLE: 0.014445536026312903
Mean Residual Deviance: 236.65292080513183
R^2: 0.9927261334349801
Null degrees of freedom: 7162
Residual degrees of freedom: 7159
Null deviance: 233128806.9562735
Residual deviance: 1695144.8717271592
AIC: 59494.93053628575

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
aic                     11904.4      98.7048      11908.5       12070.9       11863.5       11815.9       11863
loglikelihood           0            0            0             0             0             0             0
mae                     11.5871      0.165634     11.4681       11.7761       11.7596       11.4841       11.4476
mean_residual_deviance  236.65       16.4091      236.349       264.72        229.044       222.835       230.301
mse                     236.65       16.4091      236.349       264.72        229.044       222.835       230.301
null_deviance           4.66258e+07  2.682e+06    4.81296e+07   4.54785e+07   4.2432e+07    4.88948e+07   4.81938e+07
r2                      0.992697     0.000728308  0.992962      0.991658      0.992261      0.99346       0.993143
residual_deviance       339029       23583.7      338689        379344        328221        319100        329792
rmse                    15.3763      0.524225     15.3737       16.2702       15.1342       14.9277       15.1757
rmsle                   0.0143355    0.00198776   0.0177834     0.0135703     0.0127675     0.014133      0.0134236

Scoring History: 
     timestamp            duration    iteration    lambda    predictors    deviance_train      deviance_xval       deviance_se        alpha    iterations    training_rmse       training_deviance    training_mae        training_r2
---  -------------------  ----------  -----------  --------  ------------  ------------------  ------------------  -----------------  -------  ------------  ------------------  -------------------  ------------------  ------------------
     2025-07-20 15:36:14  0.000 sec   1            .17E5     4             32524.952594476155  32538.845616800107  840.611168810454   0.0
     2025-07-20 15:36:14  0.001 sec   2            .1E5      4             32519.018786591325  32534.096467138937  840.493262416532   0.0
     2025-07-20 15:36:14  0.002 sec   3        

In [12]:
# top models
aml_top3.leaderboard.head(rows=10)


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GLM_1_AutoML_2_20250720_153556,15.3835,236.653,11.5871,0.0144455,236.653
StackedEnsemble_AllModels_1_AutoML_2_20250720_153556,15.4302,238.091,11.5348,0.0347792,238.091
StackedEnsemble_BestOfFamily_1_AutoML_2_20250720_153556,15.4434,238.499,11.5145,0.0266967,238.499
XGBoost_3_AutoML_2_20250720_153556,18.3731,337.57,12.8974,0.0394762,337.57
XGBoost_2_AutoML_2_20250720_153556,19.0153,361.58,13.1755,0.0556834,361.58
GBM_3_AutoML_2_20250720_153556,19.3612,374.857,12.8427,0.0656982,374.857
DRF_1_AutoML_2_20250720_153556,19.5183,380.963,13.1432,0.0355993,380.963
GBM_4_AutoML_2_20250720_153556,19.5707,383.011,12.8606,0.0656183,383.011
GBM_2_AutoML_2_20250720_153556,19.5969,384.039,12.7446,0.0660707,384.039
XRT_1_AutoML_2_20250720_153556,19.6054,384.372,13.2215,0.039003,384.372


In [13]:
for model_id in aml_top3.leaderboard['model_id'].as_data_frame().values[:10]:
    model = h2o.get_model(model_id[0])
    runtime_ms = model._model_json['output']['run_time']
    print(f"{model.model_id} - Training time: {runtime_ms / 1000:.2f} sec")


/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


GLM_1_AutoML_2_20250720_153556 - Training time: 0.05 sec
StackedEnsemble_AllModels_1_AutoML_2_20250720_153556 - Training time: 4.61 sec
StackedEnsemble_BestOfFamily_1_AutoML_2_20250720_153556 - Training time: 4.09 sec
XGBoost_3_AutoML_2_20250720_153556 - Training time: 3.56 sec
XGBoost_2_AutoML_2_20250720_153556 - Training time: 1.28 sec
GBM_3_AutoML_2_20250720_153556 - Training time: 0.72 sec
DRF_1_AutoML_2_20250720_153556 - Training time: 2.18 sec
GBM_4_AutoML_2_20250720_153556 - Training time: 0.76 sec
GBM_2_AutoML_2_20250720_153556 - Training time: 0.64 sec
XRT_1_AutoML_2_20250720_153556 - Training time: 1.47 sec


## FLAML

In [15]:
!pip install flaml
!pip install numpy==1.24.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 41.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>

In [2]:
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = data.select_dtypes(include=["number"]).drop(columns=["Total_lift"])
y = data["Total_lift"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


### All features

In [3]:
automl = AutoML()

automl_settings = {
    "time_budget": 300,
    "metric": "rmse",
    "task": "regression",
    "log_file_name": "flaml_automl.log",
    "verbose": 1,
    "eval_method": "cv",
    "n_splits": 5,
    "log_type": "all"
}


automl.fit(X_train=X_train, y_train=y_train, **automl_settings)



INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


In [4]:
# evaluate
y_pred = automl.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print("FLAML RMSE:", rmse)

print("Best FLAML model:", automl.model.estimator)

FLAML RMSE: 13.780835653506303
Best FLAML model: LGBMRegressor(colsample_bytree=0.909434114883554,
              learning_rate=0.1845022867575554, max_bin=255,
              min_child_samples=2, n_estimators=1204, n_jobs=-1, num_leaves=4,
              reg_alpha=0.011162641168302114, reg_lambda=0.04895362477439728,
              verbose=-1)


In [5]:
import matplotlib.pyplot as plt
import numpy as np

model = automl.model.estimator

if hasattr(model, "feature_importances_"):
    importances = model.feature_importances_
    feature_names = X_train.columns

    # top 5 features
    indices = np.argsort(importances)[::-1]
    top5_idx = indices[:5]

    print("Top 5 features:")
    for i in top5_idx:
        print(f"{feature_names[i]}: {importances[i]}")
else:
    print("Feature importances not available for this model.")


Top 5 features:
deadlift: 1052
backsq: 1011
candj: 898
snatch: 540
weight: 48


In [6]:
!pip install lightgbm xgboost

In [7]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
import time

model_map = {
    'lgbm': LGBMRegressor,
    'xgboost': XGBRegressor,
    'xgb_limitdepth': XGBRegressor,
    'rf': RandomForestRegressor,
    'extra_tree': ExtraTreesRegressor,
    'sgd': SGDRegressor,
}

# clean configs
def sanitize_config(name, config):
    config = config.copy()
    if name in ['rf', 'extra_tree']:
        config.pop('max_leaves', None)
    if 'log_max_bin' in config:
        config.pop('log_max_bin')
    return config


In [8]:
results = []

for model_name, config in automl.best_config_per_estimator.items():
    print(f"\nEvaluating: {model_name}")

    ModelClass = model_map[model_name]
    config = sanitize_config(model_name, config)

    try:
        model = ModelClass(**config)
        model.fit(X_train, y_train)

        start = time.time()
        y_pred = model.predict(X_test)
        inference_time = time.time() - start
        rmse = mean_squared_error(y_test, y_pred)

        print(f"RMSE: {rmse:.2f} | Time: {inference_time:.4f}s")

        results.append({
            "model": model_name,
            "rmse": rmse,
            "inference_time": inference_time
        })

    except Exception as e:
        print(f"Skipped {model_name} due to error: {e}")



Evaluating: lgbm
RMSE: 13.78 | Time: 1.9339s

Evaluating: rf
RMSE: 142.14 | Time: 0.1761s

Evaluating: xgboost
RMSE: 139.52 | Time: 0.0483s

Evaluating: extra_tree
RMSE: 166.93 | Time: 0.0289s

Evaluating: xgb_limitdepth
RMSE: 243.54 | Time: 0.0105s

Evaluating: sgd
RMSE: 4903448334509698434295754915840.00 | Time: 0.0020s


In [9]:
# validation RMSE
top_by_rmse = sorted(results, key=lambda x: x['rmse'])[:3]

print("Top 3 Models by Validation RMSE:")
for r in top_by_rmse:
    print(f"Model: {r['model']:15s} | RMSE: {r['rmse']:.2f} ")

# inference time
print("\n Top 3 Models by Inference Speed:")
top_by_speed = sorted(results, key=lambda x: x['inference_time'])[:3]
for r in top_by_speed:
    print(f"Model: {r['model']:15s} | Time: {r['inference_time']:.4f}s ")


Top 3 Models by Validation RMSE:
Model: lgbm            | RMSE: 13.78 
Model: xgboost         | RMSE: 139.52 
Model: rf              | RMSE: 142.14 

 Top 3 Models by Inference Speed:
Model: sgd             | Time: 0.0020s 
Model: xgb_limitdepth  | Time: 0.0105s 
Model: extra_tree      | Time: 0.0289s 


### Top three features

In [10]:
top3_features = ["deadlift", "backsq", "candj"]

X = data[top3_features]
y = data["Total_lift"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# set up FLAML
automl_top3 = AutoML()

automl_settings = {
    "time_budget": 300,
    "metric": "rmse",
    "task": "regression",
    "log_file_name": "flaml_automl_top3.log",
    "verbose": 1,
    "eval_method": "cv",
    "n_splits": 5,
    "log_type": "all"
}

automl_top3.fit(X_train=X_train, y_train=y_train, **automl_settings)


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


In [12]:
results = []

for model_name, config in automl_top3.best_config_per_estimator.items():
    print(f"\nEvaluating: {model_name}")

    ModelClass = model_map[model_name]
    config = sanitize_config(model_name, config)

    try:
        model = ModelClass(**config)
        model.fit(X_train, y_train)

        start = time.time()
        y_pred = model.predict(X_test)
        inference_time = time.time() - start
        rmse = mean_squared_error(y_test, y_pred)

        print(f"RMSE: {rmse:.2f} | Time: {inference_time:.4f}s")

        results.append({
            "model": model_name,
            "rmse": rmse,
            "inference_time": inference_time
        })

    except Exception as e:
        print(f"Skipped {model_name} due to error: {e}")



Evaluating: lgbm
RMSE: 243.03 | Time: 0.6650s

Evaluating: rf
RMSE: 360.54 | Time: 0.2645s

Evaluating: xgboost
RMSE: 361.19 | Time: 0.0233s

Evaluating: extra_tree
RMSE: 323.50 | Time: 1.4953s

Evaluating: xgb_limitdepth
RMSE: 377.04 | Time: 0.0139s

Evaluating: sgd
RMSE: 182084.03 | Time: 0.0020s


In [13]:
# top by RMSE
top_by_rmse = sorted(results, key=lambda x: x['rmse'])[:3]
print("\nTop 3 Models by Validation RMSE:")
for r in top_by_rmse:
    print(f"Model: {r['model']:15s} | RMSE: {r['rmse']:.2f}")

# top by inference speed
top_by_speed = sorted(results, key=lambda x: x['inference_time'])[:3]
print("\nTop 3 Models by Inference Speed:")
for r in top_by_speed:
    print(f"Model: {r['model']:15s} | Time: {r['inference_time']:.4f}s")



Top 3 Models by Validation RMSE:
Model: lgbm            | RMSE: 243.03
Model: extra_tree      | RMSE: 323.50
Model: rf              | RMSE: 360.54

Top 3 Models by Inference Speed:
Model: sgd             | Time: 0.0020s
Model: xgb_limitdepth  | Time: 0.0139s
Model: xgboost         | Time: 0.0233s
